In [16]:
! pip install -q -U bitsandbytes
! pip install -q -U datasets
! pip install -q -U git + https :// github .com / huggingface / transformers . git
! pip install -q -U git + https :// github .com / huggingface / peft .git
! pip install -q -U git + https :// github .com / huggingface / accelerate .git
! pip install -q -U loralib
! pip install -q -U einops
! pip install -q -U googletrans==3.1.0a0
!pip install -U accelerate
!pip install -U peft


ERROR: Invalid requirement: '+'
ERROR: Invalid requirement: '+'
ERROR: Invalid requirement: '+'


In [17]:
import locale
print(locale.getpreferredencoding())

UTF-8


In [18]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [19]:
import json
import os
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers

from googletrans import Translator
from pprint import pprint
from datasets import load_dataset
from huggingface_hub import notebook_login
from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training
)
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig
)

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [34]:
MODEL_NAME = "vilm/vinallama-7b-chat"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [35]:
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [36]:
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [37]:
config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=[
        "q_proj",
        "up_proj",
        "o_proj",
        "k_proj",
        "down_proj",
        "gate_proj",
        "v_proj"
    ],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

In [38]:
model = get_peft_model(model, config)

In [39]:
generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

# Fine-tune Demo (Ignore)

In [ ]:
prompt = """
<| im_start |> system
Bạn là một trợ lý AI hữu ích. Hãy trả lời người dùng một cách chính xác.
<| im_end |>
<| im_start |> ủe
Viết hàm tính tổng hai số trong Python
<| im_end |>
<| im_start |> assistant
"""

**Run**

In [ ]:
%%time
device = "cuda" if torch.cuda.is_available() else "cpu"

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
    outputs = model.generate(
        **encoding,
        generation_config=generation_config
    )
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

 
<| im_start |> system
Bạn là một trợ lý AI hữu ích. Hãy trả lời người dùng một cách chính xác.
<| im_end |> 
<| im_start |> ủe
Viết hàm tính tổng hai số trong Python
<| im_end |> 
<| im_start |> assistant
# Hàm tính tổng của hai số 
def add two numbers (a, b): 
# Trả về tổng của a và b 
trả về a + b 
# Ví dụ 
a = 5 
b = 7 
c = add two numbers (a, b) 
print (c) # Kết xuất : 12 
<| im_end |> 
<| im_start |> 
<| im_end |> 
<| im_start |> 
<| im_end |> 
<| im_start |> 
<| im_end |> 
<| im_start |> 
<| im_end |> 
<| im_start |> 
<| im_end |> 
<| im_start |> 
<| im_end |> 
<| im_start |> 
CPU times: user 28.9 s, sys: 1.18 s, total: 30.1 s
Wall time: 33.8 s


# Data for fine-tuning

In [ ]:
data = load_dataset("alespalla/chatbot_instruction_prompts")

In [ ]:
data

DatasetDict({
    train: Dataset({
        features: ['response', 'prompt'],
        num_rows: 258042
    })
    test: Dataset({
        features: ['response', 'prompt'],
        num_rows: 64511
    })
})

In [ ]:
def generate_prompt(datapoint):
    translator = Translator()
    vn_prompt = translator.translate(datapoint["prompt"], src="en", dest="vi").text
    vn_response = translator.translate(datapoint["response"], src="en", dest="vi").text
    return f"""
<| im_start |> system
Bạn là một trợ lý AI hữu ích. Hãy trả lời người dùng một cách chính xác.
<| im_end |>
<| im_start |> ủe
{vn_prompt}
<| im_end |>
<| im_start |> assistant
{vn_response}
    """.strip()

In [ ]:
def generate_and_tokenize_prompt(datapoint):
    full_prompt = generate_prompt(datapoint)
    tokenized_full_prompt = tokenizer(full_prompt, padding=True, truncation=True)
    return tokenized_full_prompt

In [ ]:
train_data = data["train"].shard(num_shards=50, index=0) \
        .filter(lambda sample: len(sample["response"]) > 0 and len(sample["prompt"]) > 0) \
        .shuffle() \
        .map(generate_and_tokenize_prompt)

Map:   0%|          | 0/5161 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


**Fine-tuning**

In [ ]:
training_args = transformers.TrainingArguments(
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    learning_rate=2e-4,
    fp16=True,
    save_total_limit=3,
    logging_steps=1,
    output_dir="experiments",
    optim="paged_adamw_8bit",
    lr_scheduler_type="cosine",
    warmup_ratio=0.05
)

In [ ]:
trainer = transformers.Trainer(
    model=model,
    train_dataset=train_data,
    args=training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(
        tokenizer, mlm=False
    )
)
model.config.use_cache=False

In [ ]:
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
1,2.570200
2,2.660800
3,2.549800
4,2.775500
5,2.565800
6,2.121800
7,2.476500
8,2.459300
9,2.124800
10,2.107600


Step,Training Loss
1,2.570200
2,2.660800
3,2.549800
4,2.775500
5,2.565800
6,2.121800
7,2.476500
8,2.459300
9,2.124800
10,2.107600


# Fine-tuning VinaLLaMA for solving elementary math

**Dataset**

In [22]:
def generate_prompt(question, choices, explanation):
    return f"""
<| im_start | > system
Bạn là một chuyên gia về toán. Bạn sẽ nhận câu hỏi trắc nghiệm kèm theo các lựa chọn, hãy giải step by step nếu có và chọn phương án đúng.

 <| im_start | > user
 ### Câu hỏi:
 { question }
 ### Các lựa chọn:
 { choices }
 ### Câu trả lời:

 <| im_start | > assistant
 { explanation }
 """ . strip ()

In [50]:
def generate_and_tokenize_prompt ( question , choices , explanation ) :
    full_prompt = generate_prompt ( question , choices , explanation )
    tokenized_full_prompt = tokenizer (
    full_prompt ,
    padding =True ,
    truncation = True
    )
    return tokenized_full_prompt

In [14]:
data = load_dataset ("hllj/vi_grade_school_math_mcq")

Generating train split: 0 examples [00:00, ? examples/s]

In [15]:
data

DatasetDict({
    train: Dataset({
        features: ['grade', 'id', 'url', 'problems', 'title'],
        num_rows: 2733
    })
})

In [43]:
from tqdm import tqdm
from datasets import Dataset
training_samples = []

for sample in tqdm(data["train"]):
    for quest in sample["problems"]:
        choices = quest["choices"]
        explanation = quest["explanation"].strip()
        question = quest["question"]

        if explanation == "" or question == "" or choices == []:
            continue
        try:
            question = question.split("\n \n")[1].strip()
        except:
            continue

        choices = "\n".join(choices)
        training_sample = generate_and_tokenize_prompt(
            question, choices, explanation
        )

        training_samples.append(training_sample)

choices_data = Dataset.from_list(training_samples)

100%|██████████| 2733/2733 [00:03<00:00, 797.92it/s] 


In [44]:
len(training_samples)

9173

In [45]:
choices_data

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 9173
})

In [46]:
choices_data = choices_data.train_test_split(test_size=0.2)

In [47]:
choices_data

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 7338
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 1835
    })
})

training arguments

In [48]:
training_args = transformers.TrainingArguments(
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    learning_rate=2e-4,
    fp16=True,
    save_total_limit=3,
    logging_steps=1,
    output_dir="experiments",
    optim="paged_adamw_8bit",
    lr_scheduler_type="cosine",
    warmup_ratio=0.05
)

In [49]:
trainer = transformers.Trainer(
    model=model,
    train_dataset=choices_data["test"],
    args=training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
model.config.use_cache = False
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
1,3.437900
2,3.154300
3,3.406300
4,3.489200
5,3.457100
6,2.912700
7,3.093900
8,2.705800
9,2.354200
10,2.241600


TrainOutput(global_step=458, training_loss=0.6570097681736842, metrics={'train_runtime': 3127.4967, 'train_samples_per_second': 0.587, 'train_steps_per_second': 0.146, 'total_flos': 1.0808296892891136e+16, 'train_loss': 0.6570097681736842, 'epoch': 1.0})

In [53]:
question = "số gồm 1 đơn vị và 2 chục đọc là:"
choices = """
A. 20
B. 21
C. 30
D. 31
"""

prompt = f"""
<| im_start | > system
Bạn là một chuyên gia về toán. Bạn sẽ nhận câu hỏi trắc nghiệm kèm theo các lựa chọn, hãy giải step by step nếu có và chọn phương án đúng.

 <| im_start | > user
 ### Câu hỏi:
 { question }
 ### Các lựa chọn:
 { choices }
 ### Câu trả lời:

 <| im_start | > assistant

 """


In [59]:
device = "cuda" if torch.cuda.is_available() else "cpu"
encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
    outputs = model.generate(
        input_ids=encoding.input_ids,
        attention_mask=encoding.attention_mask,
        generation_config=generation_config
    )

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


In [60]:
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


<| im_start | > system
Bạn là một chuyên gia về toán. Bạn sẽ nhận câu hỏi trắc nghiệm kèm theo các lựa chọn, hãy giải step by step nếu có và chọn phương án đúng.

 <| im_start | > user
 ### Câu hỏi:
 số gồm 1 đơn vị và 2 chục đọc là:
 ### Các lựa chọn:
 
A. 20
B. 21
C. 30
D. 31

 ### Câu trả lời:

 <| im_start | > assistant
 
 
 Đáp án C 
 Số gồm 1 đơn vị và 2 chục đọc là: 30 
 Đáp án cần chọn là C. 
 Chọn C. 
 Đáp án cần chọn là: 30. 
 Chọn C. 
 Đáp án C. 
 Đáp án cần chọn là: 30. 
 Chọn C. 
 Đáp án cần chọn là: 30. 
 Chọn C. 
 Đáp án cần chọn là: 30. 
 Chọn C. 
 Đáp án cần chọn là: 30. 
 Chọn C. 
 Đáp án cần chọn là: 30. 
 Chọn C. 
 Đáp án cần chọn là: 30. 
 Chọn C. 
 Đáp án cần chọn là: 30. 
 Chọn C. 
 Đáp án cần chọn là: 30. 
 Chọn C. 
 Đáp án cần chọn là: 30. 
 Chọn C. 
 Đáp án cần chọn là: 30. 

